# Bilan Hydrique

Sources :

- Coefficients culturaux :
  - [ARDEPI](https://www.ardepi.fr/nos-services/vous-etes-irrigant/estimer-ses-besoins-en-eau/maraichage/)
  - [Chambre d’agriculture Nouvelle-Aquitaine](https://gironde.chambre-agriculture.fr/fileadmin/user_upload/Nouvelle-Aquitaine/100_Inst-Gironde/Documents/pdf_grandes-cultures_accompagnement-technique_mieux-irriguer/Messages_irrigation_2019/message_1/Tableau_Coefficients_Culturaux_Kc_.02.pdf)


In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Constantes
# Coefficients culturaux (KC) par culture et par stade
KC = {
    'Pomme de terre': {'Vegetation': 0.9, 'Maximale': 1.05}
}

# Réserve Utile (RU) par cm de terre fine (mm/cm de terre fine) en fonction de la texture du sol
RU_PAR_CM_DE_TF = {
    'Terres argileuses': 1.85,
    'Argiles sableuses': 1.7, 'Argiles sablo-limoneuses': 1.8, 'Argiles limono-sableuses': 1.8, 'Argiles limoneuses': 1.9,
    'Terres argilo-sableuses': 1.7, 'Terres argilo-limono-sableuses': 1.8, 'Terres argilo-limoneuses': 2.,
    'Terres sablo-argileuses': 1.4, 'Terres sablo-limono-argileuses': 1.5, 'Terres limono-sablo-argileuses': 1.65, 'Terres limono-argileuses': 2.00,
    'Terres sableuses': 0.7, 'Terres sablo-limoneuses': 1., 'Terres limono-sableuses': 1.55, 'Terres limoneuses': 1.8
}

# Profondeurs d'enracinement typiques
PROFONDEUR_ENRACINEMENT_TYPIQUE = {
    'Radis': 15.,
    'Salade': 15.,
    'Choux': 20.,
    'Epinard': 20.,
    'Oignon': 20.,
    'Aubergine': 30.,
    'Carotte': 30.,
    'Courge': 30.,
    'Courgette': 30.,
    'Poivron': 30.,
    'Pomme de terre': 30.,
    'Tomate': 30.
}   

In [3]:
# Données météorologiques
# Évapotranspiration Potentielle (ETP) sur la période (mm)
ETP = 28.

# Réserve Facilement Utilisable (RFU) initiale (mm)
RFU_INITIALE = 40.

# Pluviométrie (mm)
PLUVIOMETRIE = 5.

In [4]:
# RFU finale cible (mm)
RFU_FINALE_CIBLE = 40.

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_REMPLIE = 1.

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.3

In [5]:
# Choix de la culture
culture = 'Pomme de terre'

# Choix du stade
stade = 'Vegetation'

# Choix de la texture
texture = 'Terres limoneuses'

In [6]:
# RU par cm de terre fine pour cette texture
ru_par_cm_de_tf_texture = RU_PAR_CM_DE_TF[texture]

# Calcul de la profondeur de terre fine
profondeur = PROFONDEUR_ENRACINEMENT_TYPIQUE[culture]
profondeur_tf = profondeur * (1. - FRACTION_CAILLOUX)

# Calcul de la RU (mm)
ru = ru_par_cm_de_tf_texture * profondeur_tf

# Calcul de la RFU maximale (mm)
rfu_max = ru * RU_VERS_RFU

# Calcul de la RFU disponible (mm)
rfu = rfu_max * FRACTION_REMPLIE

print(f"RFU pour une profondeur d'enracinement de {profondeur:.0f} cm : {rfu:.0f} mm")

RFU pour une profondeur d'enracinement de 30 cm : 25 mm


In [7]:
# KC de la culture pour ce stade
kc_culture = KC[culture][stade]

# Calcul de l'évalotranspiration maximale (mm)
etm_culture = kc_culture * ETP

print(f'Évapotranspiration maximale pour la {culture} au stade {stade}: {etm_culture:.0f} (mm)')

Évapotranspiration maximale pour la Pomme de terre au stade Vegetation: 25 (mm)


In [8]:
# Calcul de la RFU finale
besoin_irrigation = RFU_FINALE_CIBLE + etm_culture - (rfu + PLUVIOMETRIE)

print(f'Besoin en irrigation pour la {culture} au stade {stade}: {besoin_irrigation:.0f} (mm)')

Besoin en irrigation pour la Pomme de terre au stade Vegetation: 35 (mm)


In [9]:
# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]

In [10]:
from io import StringIO
import json
import requests

# Definitions pour accéder à l'API Météo-France via un token OAuth2
# unique application id : you can find this in the curl's command to generate jwt token 
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# url to obtain acces token
TOKEN_URL = "https://portail-api.meteofrance.fr/token"

class Client(object):
    def __init__(self):
        self.session = requests.Session()
        
    def request(self, method, url, **kwargs):
        # First request will always need to obtain a token first
        if 'Authorization' not in self.session.headers:
            self.obtain_token()
            
        # Optimistically attempt to dispatch reqest
        response = self.session.request(method, url, **kwargs)

        if self.token_has_expired(response):
            # We got an 'Access token expired' response => refresh token
            self.obtain_token()

            # Re-dispatch the request that previously failed
            response = self.session.request(method, url, **kwargs)

        return response

    def token_has_expired(self, response):
        status = response.status_code
        content_type = response.headers['Content-Type']
        repJson = response.text

        if status == 401 and 'application/json' in content_type:
            repJson = response.text
            
            if 'Invalid JWT token' in repJson['description']:
                return True

        return False

    def obtain_token(self):
        # Obtain new token
        data = {'grant_type': 'client_credentials'}
        headers = {'Authorization': 'Basic ' + APPLICATION_ID}
        access_token_response = requests.post(
            TOKEN_URL, data=data, verify=False, allow_redirects=False, headers=headers)
        token = access_token_response.json()['access_token']

        # Update session with fresh token
        self.session.headers.update({'Authorization': 'Bearer %s' % token})

def response_text_to_frame(response, **read_csv_kwargs):
    return pd.read_csv(StringIO(response.text), sep=';', **read_csv_kwargs)

In [11]:
# Initialisation d'un client pour accéder à l'API Météo-France
client = Client()

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [12]:
# URL de la liste des stations
URL_LISTE_STATIONS = 'https://public-api.meteofrance.fr/public/DPObs/v1/liste-stations'

# Étiquettes de la latitude et de la longitude
LATLON_LABELS = ['Latitude', 'Longitude']

In [13]:
# Demande de la liste des stations
response_liste_stations = client.request('GET', URL_LISTE_STATIONS, verify=False)

/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portail-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [14]:
# Conversion du texte répondu en DataFrame
df_liste_stations = response_text_to_frame(response_liste_stations, index_col='Nom_usuel')

In [15]:
# Convert degrees to radians
latlon_rad_labels = []
for station_name, station_series in df_liste_stations[LATLON_LABELS].items():
    coord_label_rad = f'{station_name}_rad'
    df_liste_stations[coord_label_rad] = np.deg2rad(station_series)
    latlon_rad_labels.append(coord_label_rad)
ref_station_latlon_rad = np.deg2rad(REF_STATION_LATLON)

In [16]:
from sklearn.neighbors import BallTree

# Calcul de l'arbre des plus proches voisins pour la liste des stations
X = df_liste_stations[latlon_rad_labels].values
tree = BallTree(X, metric='haversine')

In [51]:
# Rayon de la terre (km)
RAYON_TERRE_KM = 6371.

# # Identification d'un certain nombre de plus proches voisins
# NN_NUMBER = 20
# dist_rad_arr, ind_arr = tree.query([ref_station_latlon_rad], k=NN_NUMBER)

# Alternative : identification des plus proches voisins dans un certain rayon
NN_RAYON_KM = 35.
nn_rayon_rad = NN_RAYON_KM / RAYON_TERRE_KM
ind_arr, dist_rad_arr = tree.query_radius(
    [ref_station_latlon_rad], nn_rayon_rad,
    count_only=False, return_distance=True, sort_results=True)

dist_rad, ind = dist_rad_arr[0], ind_arr[0]

# Conversion en km de la distance en rad
dist_km = np.round(dist_rad * RAYON_TERRE_KM).astype(int)

In [52]:
# Sélection des plus proches voisins
df_liste_stations_nn = df_liste_stations.iloc[ind]
s_dist_km = pd.Series(dist_km, index=df_liste_stations_nn.index, name='Distance (km)')
print(f'{REF_STATION_NAME} ({REF_STATION_LATLON[0]:.6f}, {REF_STATION_LATLON[1]:.6f}) est à :')
for k, (nn_station_name, nn_station_series) in enumerate(df_liste_stations_nn.transpose().items()):
    print(f'- {s_dist_km.loc[nn_station_name]:02d} km du {k}NN {nn_station_name} '
          f'({nn_station_series[LATLON_LABELS[0]]:.6f}, {nn_station_series[LATLON_LABELS[1]]:.6f})')

La Petite Claye (48.541356, -1.615400) est à :
- 06 km du 0NN BROUALAN (48.485667, -1.640833)
- 09 km du 1NN PONTORSON (48.585667, -1.505167)
- 17 km du 2NN PLERGUER (48.524833, -1.843667)
- 24 km du 3NN FEINS  SA (48.326833, -1.596833)
- 27 km du 4NN PLESDER (48.406833, -1.924833)
- 29 km du 5NN MEZIERES-SUR-C. (48.308833, -1.439000)
- 31 km du 6NN ST OVIN (48.682500, -1.248667)
- 33 km du 7NN GRANVILLE (48.834500, -1.613667)
- 34 km du 8NN DINARD (48.584833, -2.076333)


In [53]:
# URL de base des données horaires
URL_DONNEE_HORAIRE = 'https://public-api.meteofrance.fr/public/DPObs/v1/station/horaire'

# Format des donnée renvoyée
FMT = 'csv'

def demande_donnee_station_date(id_station, date, verify=False):
    '''Demande des données horaires pour une station et une date données.'''
    # Paramètres définissant la station, la date et le format des données
    params_station_date = {'id_station': id_station, 'date': date, 'format': FMT}

    # Demande de la liste des stations
    response_station = client.request(
        'GET', URL_DONNEE_HORAIRE, verify=verify, params=params_station_date)

    return response_station

def response_station_to_series(response_station):
    '''Conversion du texte répondu en DataFrame.'''
    df_station = response_text_to_frame(response_station)
    s_station = df_station.iloc[0]

    return s_station

def compiler_donnees_des_stations_date(df_liste_stations, date):
    df_meteo_des_stations = pd.DataFrame(dtype=float)
    for station_name in df_liste_stations.index:
        # Identifiant de la station
        id_station = int(df_liste_stations.loc[station_name]['Id_station'])
        response_station = demande_donnee_station_date(id_station, date)
        s_station = response_station_to_series(response_station)
        df_meteo_des_stations = pd.concat([
            df_meteo_des_stations, s_station.to_frame(station_name).transpose()])
    return df_meteo_des_stations

In [55]:
# Choix de la date
DATE = '2025-01-03T01:00:00Z'

df_meteo_des_stations = compiler_donnees_des_stations_date(df_liste_stations_nn, DATE)

/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/late

In [56]:
# Liste des variables à utiliser
LISTE_VARIABLES_METEO = ['ff', 't', 'u', 'ray_glo01']

# Interpolation des plus proches voisins pondérée par l'inverse de la distance au carré
weights = 1. / s_dist_km**2
df_variables_meteo_des_stations = df_meteo_des_stations[LISTE_VARIABLES_METEO]
weights_2d = (df_variables_meteo_des_stations + 1.e-6).mul(
    weights, axis='index') / (df_variables_meteo_des_stations + 1.e-6)
s_meteo_ref_station = ((df_variables_meteo_des_stations * weights_2d).sum(0) / weights_2d.sum(0)).to_frame(
    name=df_meteo_des_stations['validity_time'].iloc[0]).transpose()

                            ff           t          u ray_glo01
2025-01-03T01:00:00Z  1.578048  274.738692  91.016121       0.0
                  ff       t    u ray_glo01
BROUALAN         NaN  274.75  NaN       NaN
PONTORSON        1.4  274.35   92       NaN
PLERGUER         NaN  274.85  NaN       NaN
FEINS  SA        1.3  275.55   92       NaN
PLESDER          NaN  275.45  NaN       NaN
MEZIERES-SUR-C.  NaN  274.65  NaN       NaN
ST OVIN          NaN  274.65  NaN       NaN
GRANVILLE        4.1     NaN  NaN       NaN
DINARD           2.0  276.95   75         0


In [63]:
# Chaleur latente de vaporisation de l’eau (J kg-1)
LAMBDA = 2.45e6

# Constante psychométrique (kPa K-1)
GAMMA = 0.000665

# Constante de Stefan (W m-2 K-4)
SIGMA = 5.67e8 

# Albedo
ALPHA = 0.2

# Émissivité
EPSILON = 0.95

# ETP journalière maximale (mm j-1)
ETP_JOUR_MAX = 9.

# Rayonnement atmosphérique incident (J m-2)
RAYONNEMENT_ATMOSPHERIQUE = 0.

def calcul_etp(s_station):
    # Variable météorologiques
    # Vitesse du vent à 10 m (m s-1)
    vitesse_vent_10m = s_station['ff'].iloc[0]
    print(f'Vitesse du vent à 10 m (m s-1) : {vitesse_vent_10m:.1f}')

    # Temperature de l'air à 2 m (K)
    temperature_air_2m = s_station['t'].iloc[0]
    print(f"Temperature de l'air à 2 m (K) : {temperature_air_2m:.1f}")

    # Humidité relative de l'air à 2 m
    humidite_relative_air_2m = s_station['u'].iloc[0] / 100
    print(f"Humidité relative de l'air à 2 m : {humidite_relative_air_2m:.2f}")

    # Rayonnement global (J m-2)
    rayonnement_global = s_station['ray_glo01'].iloc[0]
    print(f'Rayonnement global (J m-2) : {rayonnement_global:.1f}')

    # Calcul de la pente de la courbe de pression de vapeur à la température moyenne de l'air (kPa K-1)
    exposant = 17.27 * (temperature_air_2m - 273.15) / (temperature_air_2m - 35.85)
    delta = 2504 * np.exp(exposant) / (temperature_air_2m - 35.85)**2

    # Calcul de rho * Cp / ra
    rho_cp_sur_ra = LAMBDA * GAMMA * 0.26 * (1. + 0.4 * vitesse_vent_10m)

    # Calcul de la pression de vapeur saturante (kPa)
    es = 0.6108 * np.exp(exposant)

    # Calcul de la pression de vapeur effective (kPa)
    ee = humidite_relative_air_2m * es

    # Calcul du rayonnement net
    rayonnement_net = ((1 - ALPHA) * rayonnement_global +
                       EPSILON * RAYONNEMENT_ATMOSPHERIQUE -
                       EPSILON * SIGMA * temperature_air_2m**4)

    # Calcul de l'ETP (mm h-1)
    denominateur = LAMBDA * (delta + GAMMA)
    etp1 = max(0, delta * rayonnement_net / denominateur)
    etp2 = max(0, rho_cp_sur_ra * (es - ee) / denominateur)
    etp = etp1 + etp2

    return etp

In [68]:
etp = calcul_etp(s_meteo_ref_station)
print(f'ETP (mm h-1) : {etp:.2e}')

Vitesse du vent à 10 m (m s-1) : 1.6
Temperature de l'air à 2 m (K) : 274.7
Humidité relative de l'air à 2 m : 0.91
Rayonnement global (J m-2) : 0.0
ETP (mm h-1) : 3.48e-04


In [ ]:
# Calcul de l'ETP journalière (mm j-1)
etp_jour = min(ETP_JOUR_MAX, etp * 24)